#DCASE2020 data download

In [1]:
import tarfile, shutil
import zipfile
import sys, os, urllib.request, tarfile, glob
import numpy as np
import cv2
import librosa
import librosa.core
import librosa.feature
import librosa.display 
import matplotlib.pyplot as plt
import random
import pickle
from data_utils import *
from train_utils import *

In [2]:
import os
import pandas as pd
import soundfile as sf 

In [3]:
import tensorflow as tf

In [5]:
print(tf.__version__)

2.1.0


In [6]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import datetime as dt
import os
import random

In [7]:
import pathlib
import matplotlib.pyplot as plt
import IPython.display as display
from PIL import Image

In [8]:
import sklearn
from sklearn.metrics import confusion_matrix
import tempfile
import matplotlib as mpl
import pandas as pd

In [9]:
downstream_task = True
evaluate_downstream = True

In [10]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [11]:
dir_keys = 'asvspoof2019/LA/ASVspoof2019_LA_cm_protocols/'
dir_path = 'ASVspoof2019_LA_train/flac/'
train_keys = os.path.join(dir_keys, 'ASVspoof2019.LA.cm.train.trn.txt')

files = pd.read_csv(train_keys, sep=" ", header=None)
files.columns = ["speaker_ID", "clip", "-", "system_ID", "label"]

In [12]:
folder_train_bonafide = './data/train/bonafide'
folder_train_spoofed = './data/train/spoof'
folder_eval_bonafide = './data/eval/bonafide'
folder_eval_spoofed = './data/eval/spoof'

In [13]:
sf.available_formats()

{'AIFF': 'AIFF (Apple/SGI)',
 'AU': 'AU (Sun/NeXT)',
 'AVR': 'AVR (Audio Visual Research)',
 'CAF': 'CAF (Apple Core Audio File)',
 'FLAC': 'FLAC (Free Lossless Audio Codec)',
 'HTK': 'HTK (HMM Tool Kit)',
 'SVX': 'IFF (Amiga IFF/SVX8/SV16)',
 'MAT4': 'MAT4 (GNU Octave 2.0 / Matlab 4.2)',
 'MAT5': 'MAT5 (GNU Octave 2.1 / Matlab 5.0)',
 'MPC2K': 'MPC (Akai MPC 2k)',
 'OGG': 'OGG (OGG Container format)',
 'PAF': 'PAF (Ensoniq PARIS)',
 'PVF': 'PVF (Portable Voice Format)',
 'RAW': 'RAW (header-less)',
 'RF64': 'RF64 (RIFF 64)',
 'SD2': 'SD2 (Sound Designer II)',
 'SDS': 'SDS (Midi Sample Dump Standard)',
 'IRCAM': 'SF (Berkeley/IRCAM/CARL)',
 'VOC': 'VOC (Creative Labs)',
 'W64': 'W64 (SoundFoundry WAVE 64)',
 'WAV': 'WAV (Microsoft)',
 'NIST': 'WAV (NIST Sphere)',
 'WAVEX': 'WAVEX (Microsoft)',
 'WVE': 'WVE (Psion Series 3)',
 'XI': 'XI (FastTracker 2)'}

In [14]:
sf.available_subtypes(format='FLAC')

{'PCM_S8': 'Signed 8 bit PCM',
 'PCM_16': 'Signed 16 bit PCM',
 'PCM_24': 'Signed 24 bit PCM'}

In [16]:
# Verify the audios are present in the corresponding folder
from IPython.display import Audio

# path to first audio in the folder
sample_audio = os.path.join(dir_path, files["clip"][0]+".flac")
# reading the audio file
clip, sample_rate = sf.read(sample_audio)
# play audio
Audio(clip, rate = sample_rate)

In [17]:
x_file = np.array(file_load(sample_audio)[0])
to_sp(x_file)

array([[-43.713593, -43.713593, -43.713593, ..., -43.713593, -43.713593,
        -33.707016],
       [-37.4869  , -43.713593, -43.713593, ..., -43.713593, -43.713593,
        -32.45814 ],
       [-21.772387, -24.54216 , -27.449846, ..., -30.094463, -32.157814,
        -25.725937],
       ...,
       [-43.713593, -43.713593, -43.713593, ..., -43.713593, -43.713593,
        -43.713593],
       [-43.713593, -43.713593, -43.713593, ..., -43.713593, -43.713593,
        -43.713593],
       [-43.713593, -43.713593, -43.713593, ..., -43.713593, -43.713593,
        -43.713593]], dtype=float32)

In [18]:
IMG_WIDTH = 224
IMG_HEIGHT = 224

In [19]:
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

In [20]:
path = 'project_sept2020/ASVAntiSpoofing/data/'
train_path = os.path.join(path, 'train', 'spoofing_attacks')
dev_path = os.path.join(path, 'dev', 'spoofing_attacks')
eval_path = os.path.join(path, 'eval', 'spoofing_attacks')

In [21]:
train_data_dir = pathlib.Path(train_path)
dev_data_dir = pathlib.Path(dev_path)
eval_data_dir = pathlib.Path(eval_path)

In [22]:
TOTAL_TRAIN = len(list(train_data_dir.glob('*/*.flac')))
TOTAL_DEV = len(list(dev_data_dir.glob('*/*.flac')))
TOTAL_EVAL = len(list(eval_data_dir.glob('*/*.flac')))
STEPS_PER_EPOCH_TRAIN = np.ceil(TOTAL_TRAIN/BATCH_SIZE)
STEPS_PER_EPOCH_DEV = np.ceil(TOTAL_DEV/BATCH_SIZE)
STEPS_PER_EPOCH_EVAL = np.ceil(TOTAL_EVAL/BATCH_SIZE)
print("Total training images:", TOTAL_TRAIN)
print("Total dev images:", TOTAL_DEV)
print("Total eval images:", TOTAL_EVAL)
print("STEPS_PER_EPOCH_TRAIN: ", STEPS_PER_EPOCH_TRAIN)
print("STEPS_PER_EPOCH_DEV:", STEPS_PER_EPOCH_DEV)
print("STEPS_PER_EPOCH_EVAL:", STEPS_PER_EPOCH_EVAL)

Total training images: 25380
Total dev images: 24844
Total eval images: 66323
STEPS_PER_EPOCH_TRAIN:  397.0
STEPS_PER_EPOCH_DEV: 389.0
STEPS_PER_EPOCH_EVAL: 1037.0


In [36]:
models_storage_directory = "ASVSpoof2019/models"
results_storage_directory = "ASVSpoof2019/results"

In [38]:
BATCH_SIZE_DOWNSTREAM = 64 #32

In [39]:
initial_learning_rate = 0.001

In [40]:
models_names = ["VGG16", "waveletCNN"]

In [41]:
features_type='HPSS_decompose_audio' #
Augmentation_type='No'+'Augmentation'
label_smoothing=False
mixup=False
alpha_mixup=1 # [0.4, 0.8, 1.0, 1.4]
attack_mixup=False
alpha_attackMixup=1 # [0.4, 0.8, 1.0, 1.4]
is_dropout=True #use by VGG only, for wavelet is always true
is_gaussian_noise=False # [True, False]
is_l1=True #False # [True, False]
dropout_value=0.3  # [0.2, 0.3, 0.4]
model_type= models_names[1] #"waveletCNN" #+"_AMLogits" #"VGG16_1channel"+"_AMLogits"

In [42]:
if features_type in ['fbanks_delta_doubledelta', 'HPSS_power', 'HPSS_stft', 'HPSS_llfb', 'HPSS_decompose_audio', 'HPSS_llfb_doubledelta']:
    num_channels = 3
    EPOCHS_DOWNSTREAM = 1200
else:
    num_channels = 1
    EPOCHS_DOWNSTREAM = 500
    
model_architecture_features = model_type + "_" + Augmentation_type + "_" + str(mixup) + "mixup_" + str(attack_mixup) + "attackMixup_" + str(label_smoothing) + "labelSmooth_" + features_type + "_" + str(EPOCHS_DOWNSTREAM) + "epochs_" + str(BATCH_SIZE_DOWNSTREAM) + "bachSize_" + str(alpha_attackMixup).replace(".", "p") + "alphaAttackMixup_" + str(alpha_mixup).replace(".", "p") + "alphaMixup_" + str(is_dropout) + "isDropout_" + str(is_gaussian_noise) + "isGaussianNoise_" + str(is_l1) + "isL1_" + str(dropout_value).replace(".", "p") + "dropoutValue" + "_factorized"
print(model_architecture_features)

waveletCNN_NoAugmentation_Falsemixup_FalseattackMixup_FalselabelSmooth_HPSS_decompose_audio_1200epochs_64bachSize_1alphaAttackMixup_1alphaMixup_TrueisDropout_FalseisGaussianNoise_TrueisL1_0p3dropoutValue_factorized


In [44]:
if downstream_task or evaluate_downstream:
    folder_train_bonafide = './data/train/bonafide/'    
    #Bonafide data
    x_train_normal = make_data(folder_train_bonafide + "/*")
    
    x_train_normal = to_img(x_train_normal, features=features_type)

    max_ = np.max(x_train_normal)
    min_ = np.min(x_train_normal)
    print("#min, max: ", min_, max_)

    x_train_normal = (x_train_normal-min_)/(max_-min_)

    mean = np.mean(x_train_normal)
    sigma = np.std(x_train_normal)
    print("#mean, sigma: ", mean, sigma)

    x_train_normal = (x_train_normal*255-mean)/sigma

    if features_type not in ['fbanks_delta_doubledelta', 'HPSS_power', 'HPSS_stft', 'HPSS_llfb', 'HPSS_decompose_audio', 'HPSS_llfb_doubledelta']:
        x_train_normal = np.expand_dims(x_train_normal, axis=-1)
    print("#x_train_normal.shape: ", x_train_normal.shape)

/lu/fast_scratch/patx/fathanab/project_sept2020/ASVAntiSpoofing/data_utils.py:123: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(result)


#min, max:  -36.043655 5.233303
#mean, sigma:  0.80499583 0.059110437
#x_train_normal.shape:  (2580, 224, 224, 3)


In [45]:
if downstream_task:
    
    #Spoof data
    x_train_anomaly = make_data(folder_train_spoofed + "/*")
    x_train_anomaly = to_img(x_train_anomaly, features=features_type)

    x_train_anomaly = (x_train_anomaly-min_)/(max_-min_)

    x_train_anomaly = (x_train_anomaly*255-mean)/sigma

    if features_type not in ['fbanks_delta_doubledelta', 'HPSS_power', 'HPSS_stft', 'HPSS_llfb', 'HPSS_decompose_audio', 'HPSS_llfb_doubledelta']:
        x_train_anomaly = np.expand_dims(x_train_anomaly, axis=-1)
    print("#x_train_anomaly.shape: ", x_train_anomaly.shape)

#x_train_anomaly.shape:  (22800, 224, 224, 3)


##Experiment

In [ ]:
model_path_downstream = "savedModel_" + str(EPOCHS_DOWNSTREAM) + "_" + str(BATCH_SIZE_DOWNSTREAM) + "_" + model_architecture_features + "/"
model_path_downstream = os.path.join(models_storage_directory, model_path_downstream)
if not os.path.exists(model_path_downstream):
    os.makedirs(model_path_downstream)
print(model_path_downstream)

In [49]:
from models_utils import *
if downstream_task:

    def create_model():
        if model_type == "VGG16":
            return create_model_VGG16_custom_3_l1Reg_noise_dropout(IMG_WIDTH, IMG_HEIGHT, optimizer='Adam', embedding_dim=512, num_channels=num_channels, is_dropout=is_dropout, is_gaussian_noise=is_gaussian_noise, is_l1=is_l1, dropout_value=dropout_value)
        elif model_type == "waveletCNN":
            return create_model_wavelet_cnn_l1Reg_noise_dropout(IMG_WIDTH, IMG_HEIGHT, optimizer='Adam', embedding_dim=512, num_channels=num_channels, wavelet_transform=True, is_gaussian_noise=is_gaussian_noise, is_l1=is_l1, dropout_value=dropout_value)

    model = create_model()
    model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
wavelet (Lambda)                [(None, 112, 112, 12 0           the_input[0][0]                  
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 112, 112, 64) 6976        wavelet[0][0]                    
__________________________________________________________________________________________________
norm_1 (BatchNormalization)     (None, 112, 112, 64) 256         conv_1[0][0]                     
______________________________________________________________________________________________

In [50]:
if downstream_task:
    model = create_model_AM_logits(model)
    model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
wavelet (Lambda)                [(None, 112, 112, 12 0           the_input[0][0]                  
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 112, 112, 64) 6976        wavelet[0][0]                    
__________________________________________________________________________________________________
norm_1 (BatchNormalization)     (None, 112, 112, 64) 256         conv_1[0][0]                     
____________________________________________________________________________________________

In [51]:
if downstream_task:
    train_CNN(model, x_train_normal, x_train_anomaly, model_path=model_path_downstream, batch_size=BATCH_SIZE_DOWNSTREAM, num_epochs=EPOCHS_DOWNSTREAM, initial_learning_rate=initial_learning_rate, arcFace=True, label_smoothing=label_smoothing, mixup=mixup, attack_mixup=attack_mixup, alpha_attackMixup=alpha_attackMixup, alpha_mixup=alpha_mixup)

#####LATEST
training...
128/128 [==============================] - 0s 1ms/sample - loss: 9.3034 - accuracy: 0.5000
epoch: 20
Training accuracy: 0.5
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: /misc/scratch09/PATR/fathanab/ASVSpoof2019/models/savedModel_1200_64_waveletCNN_NoAugmentation_Falsemixup_FalseattackMixup_FalselabelSmooth_HPSS_decompose_audio_1200epochs_64bachSize_1alphaAttackMixup_1alphaMixup_TrueisDropout_FalseisGaussianNoise_TrueisL1_0p3dropoutValue_factorized/20_epochs/assets


INFO:tensorflow:Assets written to: /misc/scratch09/PATR/fathanab/ASVSpoof2019/models/savedModel_1200_64_waveletCNN_NoAugmentation_Falsemixup_FalseattackMixup_FalselabelSmooth_HPSS_decompose_audio_1200epochs_64bachSize_1alphaAttackMixup_1alphaMixup_TrueisDropout_FalseisGaussianNoise_TrueisL1_0p3dropoutValue_factorized/20_epochs/assets


128/128 [==============================] - 0s 986us/sample - loss: 13.6059 - accuracy: 0.5000
epoch: 40
Training accuracy: 0.5
INFO:tensorflow:Assets written to: /misc/scratch09/PATR/fathanab/ASVSpoof2019/models/savedModel_1200_64_waveletCNN_NoAugmentation_Falsemixup_FalseattackMixup_FalselabelSmooth_HPSS_decompose_audio_1200epochs_64bachSize_1alphaAttackMixup_1alphaMixup_TrueisDropout_FalseisGaussianNoise_TrueisL1_0p3dropoutValue_factorized/40_epochs/assets


...

128/128 [==============================] - 0s 973us/sample - loss: 0.0490 - accuracy: 1.0000
epoch: 1120
Training accuracy: 1.0
INFO:tensorflow:Assets written to: /misc/scratch09/PATR/fathanab/ASVSpoof2019/models/savedModel_1200_64_waveletCNN_NoAugmentation_Falsemixup_FalseattackMixup_FalselabelSmooth_HPSS_decompose_audio_1200epochs_64bachSize_1alphaAttackMixup_1alphaMixup_TrueisDropout_FalseisGaussianNoise_TrueisL1_0p3dropoutValue_factorized/1120_epochs/assets


INFO:tensorflow:Assets written to: /misc/scratch09/PATR/fathanab/ASVSpoof2019/models/savedModel_1200_64_waveletCNN_NoAugmentation_Falsemixup_FalseattackMixup_FalselabelSmooth_HPSS_decompose_audio_1200epochs_64bachSize_1alphaAttackMixup_1alphaMixup_TrueisDropout_FalseisGaussianNoise_TrueisL1_0p3dropoutValue_factorized/1120_epochs/assets


128/128 [==============================] - 0s 915us/sample - loss: 0.0425 - accuracy: 1.0000
epoch: 1140
Training accuracy: 1.0
INFO:tensorflow:Assets written to: /misc/scratch09/PATR/fathanab/ASVSpoof2019/models/savedModel_1200_64_waveletCNN_NoAugmentation_Falsemixup_FalseattackMixup_FalselabelSmooth_HPSS_decompose_audio_1200epochs_64bachSize_1alphaAttackMixup_1alphaMixup_TrueisDropout_FalseisGaussianNoise_TrueisL1_0p3dropoutValue_factorized/1140_epochs/assets


INFO:tensorflow:Assets written to: /misc/scratch09/PATR/fathanab/ASVSpoof2019/models/savedModel_1200_64_waveletCNN_NoAugmentation_Falsemixup_FalseattackMixup_FalselabelSmooth_HPSS_decompose_audio_1200epochs_64bachSize_1alphaAttackMixup_1alphaMixup_TrueisDropout_FalseisGaussianNoise_TrueisL1_0p3dropoutValue_factorized/1140_epochs/assets


 32/128 [======>.......................] - ETA: 0s - loss: 0.0393 - accuracy: 1.0000

In [52]:
if downstream_task:
    #save model
    model.save(model_path_downstream)

INFO:tensorflow:Assets written to: /misc/scratch09/PATR/fathanab/ASVSpoof2019/models/savedModel_1200_64_waveletCNN_NoAugmentation_Falsemixup_FalseattackMixup_FalselabelSmooth_HPSS_decompose_audio_1200epochs_64bachSize_1alphaAttackMixup_1alphaMixup_TrueisDropout_FalseisGaussianNoise_TrueisL1_0p3dropoutValue_factorized/assets


INFO:tensorflow:Assets written to: /misc/scratch09/PATR/fathanab/ASVSpoof2019/models/savedModel_1200_64_waveletCNN_NoAugmentation_Falsemixup_FalseattackMixup_FalselabelSmooth_HPSS_decompose_audio_1200epochs_64bachSize_1alphaAttackMixup_1alphaMixup_TrueisDropout_FalseisGaussianNoise_TrueisL1_0p3dropoutValue_factorized/assets


In [53]:
if downstream_task:
    del x_train_normal
    del x_train_anomaly

In [54]:
#Load model
if evaluate_downstream:
    model = tf.keras.models.load_model(model_path_downstream)
    # Check its architecture
    model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, 224, 224, 3)]     0         
_________________________________________________________________
wavelet (Lambda)             multiple                  0         
_________________________________________________________________
conv_1 (Conv2D)              multiple                  6976      
_________________________________________________________________
norm_1 (BatchNormalization)  multiple                  256       
_________________________________________________________________
relu_1 (Activation)          multiple                  0         
_________________________________________________________________
conv_1_2 (Conv2D)            multiple                  36928     
_________________________________________________________________
conv_a (Conv2D)              multiple                  6976

## dev set

In [55]:
if evaluate_downstream:
    folder_dev_bonafide = './data/dev/bonafide'
    folder_dev_spoofed = './data/dev/spoof'

    x_dev_normal = make_data(folder_dev_bonafide + "/*")
    x_dev_anomaly = make_data(folder_dev_spoofed + "/*")

/lu/fast_scratch/patx/fathanab/project_sept2020/ASVAntiSpoofing/data_utils.py:123: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(result)


In [56]:
if evaluate_downstream:
    X_dev_total, y_dev_total = create_data_downstream_labels(x_dev_normal, x_dev_anomaly, max_, min_, mean, sigma, features=features_type)
    y_dev_total_categ = keras.utils.to_categorical(y_dev_total)
    TOTAL_DEV = len(X_dev_total)
    STEPS_PER_EPOCH_DEV = np.ceil(TOTAL_DEV/BATCH_SIZE)

In [57]:
if evaluate_downstream:
    print("#X_dev_total.shape: " + str(X_dev_total.shape))
    print("#y_dev_total.shape: " + str(y_dev_total.shape))
    print("#y_dev_total_categ.shape: " + str(y_dev_total_categ.shape))

#X_dev_total.shape: (24844, 224, 224, 3)
#y_dev_total.shape: (24844,)
#y_dev_total_categ.shape: (24844, 2)


In [58]:
if evaluate_downstream:
    # Evaluate the model on the dev data using `evaluate`
    print("Evaluate on dev data")
    dev_results = model.evaluate([X_dev_total, y_dev_total_categ], y_dev_total_categ)#, steps=STEPS_PER_EPOCH_DEV)#batch_size=32)
    print("dev loss, dev acc:", dev_results)

    # Generate predictions (probabilities -- the output of the last layer)
    # on new data using `predict`
    print("Generate predictions")
    dev_predictions = model.predict([X_dev_total, y_dev_total_categ])#, steps=STEPS_PER_EPOCH_DEV)
    print("predictions shape:", dev_predictions.shape)

Evaluate on dev data
24844/24844 [==============================] - 22s 889us/sample - loss: 0.0344 - accuracy: 0.9997
dev loss, dev acc: [0.034391973554521844, 0.99971825]
Generate predictions
predictions shape: (24844, 2)


In [59]:
if evaluate_downstream:
    dev_predictions

In [60]:
if evaluate_downstream:
    for name, value in zip(model.metrics_names, dev_results):
        print(name, ': ', value)

loss :  0.034391973554521844
accuracy :  0.99971825


In [61]:
if evaluate_downstream:
    dev_EER = compute_eer(y_dev_total, dev_predictions[:,1])
    print(dev_EER)

0.0008073549035412843


In [62]:
if evaluate_downstream:
    del X_dev_total
    del y_dev_total
    del x_dev_normal
    del x_dev_anomaly
    del y_dev_total_categ
    del dev_results
    del dev_predictions
    del dev_EER

## test set

In [63]:
if evaluate_downstream:
    folder_eval_bonafide = './data/eval/bonafide'
    folder_eval_spoofed = './data/eval/spoof'
    x_test_normal = make_data(folder_eval_bonafide + "/*")
    x_test_anomaly = make_data(folder_eval_spoofed + "/*")

In [64]:
if evaluate_downstream:
    X_test_total, y_test_total = create_data_downstream_labels(x_test_normal, x_test_anomaly, max_, min_, mean, sigma, features=features_type)
    y_test_total_categ = keras.utils.to_categorical(y_test_total)
    TOTAL_EVAL = len(X_test_total)
    STEPS_PER_EPOCH_EVAL = np.ceil(TOTAL_EVAL/BATCH_SIZE)

In [65]:
if evaluate_downstream:
    # Evaluate the model on the test data using `evaluate`
    print("Evaluate on test data")
    test_results = model.evaluate([X_test_total, y_test_total_categ], y_test_total_categ)#, steps=STEPS_PER_EPOCH_EVAL)#batch_size=32)
    print("test loss, test acc:", test_results)

    # Generate predictions (probabilities -- the output of the last layer)
    # on new data using `predict`
    print("Generate predictions")
    test_predictions = model.predict([X_test_total, y_test_total_categ])#, steps=STEPS_PER_EPOCH_EVAL)
    print("predictions shape:", test_predictions.shape)

Evaluate on test data
71237/71237 [==============================] - 60s 844us/sample - loss: 1.7715 - accuracy: 0.8736
test loss, test acc: [1.7715182059073864, 0.8735629]
Generate predictions
predictions shape: (71237, 2)


In [66]:
if evaluate_downstream:
    test_predictions

In [67]:
if evaluate_downstream:
    for name, value in zip(model.metrics_names, test_results):
        print(name, ': ', value)

loss :  1.7715182059073864
accuracy :  0.8735629


In [68]:
if evaluate_downstream:
    test_EER = compute_eer(y_test_total, test_predictions[:,1])
    print(test_EER)

0.04813403094007602


In [69]:
if evaluate_downstream:
    import glob
    import numpy as np
    test_filenames_bonafide = [ file.split("/")[-1] for file in glob.glob(folder_eval_bonafide + "/*")]
    test_filenames_spoof = [ file.split("/")[-1] for file in glob.glob(folder_eval_spoofed + "/*")]
    test_filenames = np.concatenate((test_filenames_bonafide, test_filenames_spoof))


    test_score_file = model_architecture_features + "_test_scores_ASVSPoof2019.txt"
    test_score_file = os.path.join(results_storage_directory, test_score_file)
    store_file(test_score_file, test_filenames, test_predictions[:,0])

DONE.


In [70]:
if evaluate_downstream:
    del X_test_total
    del y_test_total
    del x_test_normal
    del x_test_anomaly
    del y_test_total_categ
    del test_results
    del test_predictions
    del test_EER

## train set

In [71]:
if evaluate_downstream:
    x_train_normal = make_data(folder_train_bonafide + "/*")
    x_train_anomaly = make_data(folder_train_spoofed + "/*")

In [72]:
if evaluate_downstream:
    X_train_total, y_train_total = create_data_downstream_labels(x_train_normal, x_train_anomaly, max_, min_, mean, sigma, features=features_type)
    y_train_total_categ = keras.utils.to_categorical(y_train_total)
    TOTAL_TRAIN = len(X_train_total)
    STEPS_PER_EPOCH_TRAIN = np.ceil(TOTAL_TRAIN/BATCH_SIZE)

In [73]:
if evaluate_downstream:
    # Evaluate the model on the dev data using `evaluate`
    print("Evaluate on train data")
    train_results = model.evaluate([X_train_total, y_train_total_categ], y_train_total_categ)#, steps=STEPS_PER_EPOCH_TRAIN)#batch_size=32)
    print("train loss, train acc:", train_results)

    # Generate predictions (probabilities -- the output of the last layer)
    # on new data using `predict`
    print("Generate predictions")
    train_predictions = model.predict([X_train_total, y_train_total_categ])#, steps=STEPS_PER_EPOCH_TRAIN)
    print("predictions shape:", train_predictions.shape)

Evaluate on train data
25380/25380 [==============================] - 22s 871us/sample - loss: 0.0304 - accuracy: 1.0000
train loss, train acc: [0.03035895526409149, 1.0]
Generate predictions
predictions shape: (25380, 2)


In [74]:
if evaluate_downstream:
    train_predictions

In [75]:
if evaluate_downstream:
    for name, value in zip(model.metrics_names, train_results):
        print(name, ': ', value)

loss :  0.03035895526409149
accuracy :  1.0


In [76]:
if evaluate_downstream:
    train_EER = compute_eer(y_train_total, train_predictions[:,1])
    print(train_EER)

0.0


In [77]:
if evaluate_downstream:
    del X_train_total
    del y_train_total
    del x_train_normal
    del x_train_anomaly
    del y_train_total_categ
    del train_results
    del train_predictions
    del train_EER